# 09. Comparison of Model System Scales

## Numerical experiments - Week 04/2025

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

## Prerequisite libraries

In [7]:
using PyPlot
using Revise

Revise.includet("./conversions.jl")
Revise.includet("./diffusion.jl")
using .Conversions
using .Diffusion

## 1. Revision of scale assumptions - 3 options

The diffusion model developed so far is based on the discretisation of space using a single spore diameter as a spatial increment. This leads to a straightforward abstraction of the spore as a special node on the lattice which differs in its concentration contribution from the nodes of the extracellular medium. However, it could be argued that this discretisation is too arbitrary, too rough or leading to incoherences in the interpretation of the interface between spore and medium. For this reason, two additional alternative schemes are proposed.

First, inspired by the approach taken in the derivation of the analytical scheme in Notebook 7, it can be assumed that the spore source is of negligible size compared to the lattice resolution and that a single lattice site is a discrete box that _contains_ the spore, but their boundaries are not coincident. The spore can therefore be seen as a singular source which is not significantly large to pose an obstacle for diffusion within the box, but adds new concentration to the box with a rate based on the concentration difference between the spore and the medium. In this sense, the slow-permeating sub-system of the spore is spatially removed from the sub-system of the medium lattice, but they do communicate via the entry node assumed to contain the spore. The concentration just outside the spore, in the container box, is then used as a reference for inhibition.

A second option considers the opposite scenario --- a much more fine-grain lattice, such that the detail of the spore's spherical shape can be captured. This becomes important in observing the spatial articulation of spore packings caused by aggregation and furthermore improves the accuracy of the diffusion simulation by considering scales closer to the infinitessimal limit.

## 2. Comparative experiment

The two newly proposed options are constructed and tested against the original discretisation scheme, applying different resolutions on a lattice with the same spatial dimensions and simulating inhibitor diffusion with a single spore. The concentration at the spore is recorded and compared to the analytical solution and among the numerical schemes.

In [19]:
GC.gc()

In [68]:
# Global parameters
D = 600 # microns^2/s
Ps_fit = 0.0002675353069511818 # microns/s
# V_spore = 125 # microns^3
A_spore = 150 # microns^2
# A_spore_sphere = 4 * π * (3 * V_spore / (4 * π))^(2/3)
V_spore_sphere = 4π * √(A_spore / (4π)) ^ 3 / 3
t_max = 1#4400 # s
c0 = 1.018 # M
spore_diameter = 5 # microns

# ----- Analytical solution -----
c_out = 0.0 # M
times = LinRange(0, t_max, 1000)

# Cube spore
V_spore_cube = spore_diameter^3 # microns^3
A_spore_cube = 6 * spore_diameter^2 # microns^2
c_analytical_cube = permeation_time_dependent_analytical(c0, c_out, times, Ps_fit, A_spore_cube, V_spore_cube)
println("Analytical solution for spore with V=$V_spore_cube and A=$A_spore_cube: ", c_analytical_cube[end])

# Sphere spore
V_spore_sphere = 4π * (spore_diameter / 2)^3 / 3 # microns^3
A_spore_sphere = 4π * (spore_diameter / 2)^2 # microns^2
c_analytical_sphere = permeation_time_dependent_analytical(c0, c_out, times, Ps_fit, A_spore_sphere, V_spore_sphere)
println("Analytical solution for spore with V=$V_spore_sphere and A=$A_spore_sphere: ", c_analytical_sphere[end])

# ----- Numerical solution -----
n_save_frames = 100

# Mid resolution
N = 64
dt = 0.005
spore_idx = (N ÷ 2, N ÷ 2, N ÷ 2)
c_init = zeros(N, N, N)
c_init[spore_idx...] = c0
c_numerical, times_numerical, t_thresholds = diffusion_time_dependent_GPU(c_init, t_max; D, Ps=Ps_fit, dt=dt, dx=spore_diameter, spore_idx=spore_idx);

c_numerical[end, spore_idx...]
maximum(c_numerical[end, :, :, :])

Analytical solution for spore with V=125 and A=150: 1.0176732313251298
Analytical solution for spore with V=65.44984694978736 and A=78.53981633974483: 1.0176732313251298
3D simulation
Using D = 600, Db = 0.0013376765347559091, Ps = 0.0002675353069511818
Storage arrays allocated.
Frame 2 saved.
Frame 3 saved.
Frame 4 saved.
Frame 5 saved.
Frame 6 saved.
Frame 7 saved.
Frame 8 saved.
Frame 9 saved.
Frame 10 saved.
Frame 11 saved.
Frame 12 saved.
Frame 13 saved.
Frame 14 saved.
Frame 15 saved.
Frame 16 saved.
Frame 17 saved.
Frame 18 saved.
Frame 19 saved.
Frame 20 saved.
Frame 21 saved.
Frame 22 saved.
Frame 23 saved.
Frame 24 saved.
Frame 25 saved.
Frame 26 saved.
Frame 27 saved.
Frame 28 saved.
Frame 29 saved.
Frame 30 saved.
Frame 31 saved.
Frame 32 saved.
Frame 33 saved.
Frame 34 saved.
Frame 35 saved.
Frame 36 saved.
Frame 37 saved.
Frame 38 saved.
Frame 39 saved.
Frame 40 saved.
Frame 41 saved.
Frame 42 saved.
Frame 43 saved.
Frame 44 saved.
Frame 45 saved.
Frame 46 saved.
Frame 47

1.017666220664978